In [239]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

# scrap du classement general TOP 14

In [240]:
# je stocke l'url de la page à scraper dans une variable avec requests.get
url = requests.get("https://top14.lnr.fr/classement")

In [241]:
# j'affiche la réponse, si 200 = OK
url

<Response [200]>

In [242]:
# j'utilise .text sur ma variable afin d'afficher son contenu
print(url.text)

<!DOCTYPE html>
<html lang="fr">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" />
  <meta name="csrf-token" content="aDXKiCirs6YxRn8RuSGnp7BrD9LHMX6L93Ce8BSI" />

  <title>Classement Top 14 | Top 14 - Site Officiel</title>
  <meta name="title" content="Classement Top 14 | Top 14 - Site Officiel" />
  <meta name="description" content="Découvrez le classement du championnat de Top 14 de Rugby" />
      <meta property="og:type" content="website" />
      <meta property="og:site_name" content="Ligue Nationale de Rugby" />
      <meta property="og:locale" content="fr" />
      <meta property="og:url" content="https://top14.lnr.fr/classement" />
      <meta property="og:title" content="Classement Top 14 | Top 14 - Site Officiel" />
      <meta property="og:description" content="Découvrez le classement du championnat de Top 14 de Rugby" />
      <meta property="fb:app_id" content="488329855239332" />
 

In [243]:
# je crée une variable soup surn laquelle j'applique BeautifulSoup
# avec en paramètre url.text et je découpe avec html.parser
soup = BeautifulSoup(url.text, 'html.parser')

In [244]:
soup

<!DOCTYPE html>

<html lang="fr">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" name="viewport"/>
<meta content="aDXKiCirs6YxRn8RuSGnp7BrD9LHMX6L93Ce8BSI" name="csrf-token"/>
<title>Classement Top 14 | Top 14 - Site Officiel</title>
<meta content="Classement Top 14 | Top 14 - Site Officiel" name="title"/>
<meta content="Découvrez le classement du championnat de Top 14 de Rugby" name="description"/>
<meta content="website" property="og:type"/>
<meta content="Ligue Nationale de Rugby" property="og:site_name"/>
<meta content="fr" property="og:locale"/>
<meta content="https://top14.lnr.fr/classement" property="og:url"/>
<meta content="Classement Top 14 | Top 14 - Site Officiel" property="og:title"/>
<meta content="Découvrez le classement du championnat de Top 14 de Rugby" property="og:description"/>
<meta content="488329855239332" property="fb:app_id"/>
<meta content="summary_large_image" property="twitter:card"/>
<meta

- scraper le nom des equipes pour constituer une liste

In [245]:
# je recherche les noms de toutes les équipes avec la balise a et la classe affichée sur la page internet
list_nom_equipes = soup.find_all('a', {'class': 'base-link base-link--black'})

In [246]:
# je vérifie que les noms des équipes sont bien dans ma liste
list_nom_equipes

[<a class="base-link base-link--black" href="https://top14.lnr.fr/club/bordeaux-begles"> Union Bordeaux-Bègles</a>,
 <a class="base-link base-link--black" href="https://top14.lnr.fr/club/toulouse"> Stade Toulousain</a>,
 <a class="base-link base-link--black" href="https://top14.lnr.fr/club/toulon"> RC Toulon</a>,
 <a class="base-link base-link--black" href="https://top14.lnr.fr/club/clermont"> ASM Clermont</a>,
 <a class="base-link base-link--black" href="https://top14.lnr.fr/club/bayonne"> Aviron Bayonnais</a>,
 <a class="base-link base-link--black" href="https://top14.lnr.fr/club/la-rochelle"> Stade Rochelais</a>,
 <a class="base-link base-link--black" href="https://top14.lnr.fr/club/montpellier"> Montpellier Hérault Rugby</a>,
 <a class="base-link base-link--black" href="https://top14.lnr.fr/club/castres"> Castres Olympique</a>,
 <a class="base-link base-link--black" href="https://top14.lnr.fr/club/racing-92"> Racing 92</a>,
 <a class="base-link base-link--black" href="https://top14

In [247]:
# html = '<a class="base-link base-link--black" href="https://top14.lnr.fr/club/bordeaux-begles"> Union Bordeaux-Bègles</a>'
# texte = re.search(r'>([^<]+)<', html).group(1).strip()
# print(texte)

In [248]:
list_team = []
for ele in list_nom_equipes:
    ele = str(ele)
    # Appliquer la regex à chaque élément `ele`
    match = re.search(r'>([^<]+)<', ele)
    if match:  # Vérifier si la regex a trouvé un résultat
        list_team.append(match.group(1).strip())

# Afficher le résultat final
print(list_team)

['Union Bordeaux-Bègles', 'Stade Toulousain', 'RC Toulon', 'ASM Clermont', 'Aviron Bayonnais', 'Stade Rochelais', 'Montpellier Hérault Rugby', 'Castres Olympique', 'Racing 92', 'LOU Rugby', 'Section Paloise', 'USA Perpignan', 'Stade Français Paris', 'RC Vannes']


- scraper le total des points des equipes pour constituer une liste

In [249]:
points_equipes = soup.find_all('div', {'class': 'table-line__cell--small'})

In [250]:
points_equipes[1]

<div class="table-line__cell--small">
                                                                                      47
                                    </div>

In [251]:
str(points_equipes[0])

'<div class="table-line__cell--small">\n                                                                                      50\n                                    </div>'

In [252]:
regex_points = re.search(r'[0-9][0-9]', str(points_equipes[0]))
regex_points[0]

'50'

In [253]:
# FONCTIONNE POUR GENERAL ET DOMICILE

# list_points_team = []
# for points in points_equipes:
#     # print(points)
#     points = str(points)
#     regex_points = re.search(r'[0-9][0-9]', points)
#     # print(regex_points)
#     regex_points[0]
#     # print(regex_points[0])
#     list_points_team.append(regex_points[0])

# print(list_points_team)


In [254]:
list_points_team = []
for points in points_equipes:
    # print(points)
    points = str(points)
    regex_points = re.search(r'[0-9]+', points)
    # print(regex_points)
    # regex_points[0]
    # print(regex_points[0])
    list_points_team.append(regex_points[0])

print(list_points_team)


['50', '47', '43', '38', '38', '36', '34', '31', '26', '26', '25', '24', '23', '16']


In [255]:
# regex_points = re.search(r'>\s*(\d+)\s*<', points[0])
# regex_points

- scraper le nombre de matchs et les mettre dans une lsite

# FIND_ALL TOUTES INFOS SUR EQUIPES

In [256]:
all_team = soup.find_all('div', {'class': 'table-line table-line--ranking-full table-line--ranking-scrollable'})

In [257]:
all_team

[<div class="table-line table-line--ranking-full table-line--ranking-scrollable">
 <div class="table-line__cell-wrapper table-line__cell-wrapper--full table-line__cell-wrapper--club-name">
 <div class="">
 <a class="base-link base-link--black" href="https://top14.lnr.fr/club/bordeaux-begles"> Union Bordeaux-Bègles</a>
 </div>
 </div>
 <div class="table-line__cell-wrapper table-line__cell-wrapper--small">
 <div class="table-line__cell--small">
                                                                                       50
                                     </div>
 </div>
 <div class="table-line__cell-wrapper table-line__cell-wrapper--small">
 <div class="">
                                                                                       14
                                     </div>
 </div>
 <div class="table-line__cell-wrapper table-line__cell-wrapper--small">
 <div class="">
                                                                                       11
   

In [258]:
# FONCTIONNE 
# RECUPERE LE NB DE POINTS D UNE EQUIPE

points = all_team[0].find_all('div', {'class': 'table-line__cell--small'})[0]
re.search(r'[0-9]+', str(points))[0]

'50'

In [259]:
# FONCTIONNE
# ITERE SUR ALL_TEAM ==> RECUPERE VIA [] PREMIERE LIGNE LES INFOS GAGNE, NUL, PERDU 

points_test = all_team[1].find_all('div', {'class': ''})[3]
re.search(r'[0-9]+', str(points_test))[0]

'1'

In [260]:
# FONCTIONNE POUR UNE EQUIPE

for data in range(1, 5):
    points_test = all_team[0].find_all('div', {'class': ''})[data]
    print(re.search(r'[0-9]+', str(points_test))[0])


14
11
0
3


In [261]:
list_matchs_joues = []
list_matchs_gagnes = []
list_matchs_nuls = []
list_matchs_perdus = []

for team in range(0, 14):
    for data in range(1, 5):
        if data == 1:
            points_test = all_team[team].find_all('div', {'class': ''})[data]
            # print(re.search(r'[0-9]+', str(points_test))[0])
            list_matchs_joues.append(re.search(r'[0-9]+', str(points_test))[0])
        elif data == 2:
            points_test = all_team[team].find_all('div', {'class': ''})[data]
            # print(re.search(r'[0-9]+', str(points_test))[0])
            list_matchs_gagnes.append(re.search(r'[0-9]+', str(points_test))[0])
        elif data == 3:
            points_test = all_team[team].find_all('div', {'class': ''})[data]
            # print(re.search(r'[0-9]+', str(points_test))[0])
            list_matchs_nuls.append(re.search(r'[0-9]+', str(points_test))[0])
        else:
            points_test = all_team[team].find_all('div', {'class': ''})[data]
            # print(re.search(r'[0-9]+', str(points_test))[0])
            list_matchs_perdus.append(re.search(r'[0-9]+', str(points_test))[0])
    # print('---')

print(list_matchs_joues)
print(list_matchs_gagnes)
print(list_matchs_nuls)
print(list_matchs_perdus)

['14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14', '14']
['11', '9', '9', '8', '9', '8', '7', '7', '5', '5', '5', '5', '5', '3']
['0', '1', '0', '0', '0', '0', '0', '0', '1', '2', '0', '0', '0', '0']
['3', '4', '5', '6', '5', '6', '7', '7', '8', '7', '9', '9', '9', '11']


In [262]:
df_test = pd.DataFrame({
    "Equipes": list_team,
    "Pts": list_points_team,
    "Matchs joués": list_matchs_joues,
    "Gagnés": list_matchs_gagnes,
    "Nuls": list_matchs_nuls,
    "Perdus": list_matchs_perdus
})

In [263]:
df_test

,Equipes,Pts,Matchs joués,Gagnés,Nuls,Perdus
0,Union Bordeaux-Bègles,50,14,11,0,3
1,Stade Toulousain,47,14,9,1,4
2,RC Toulon,43,14,9,0,5
3,ASM Clermont,38,14,8,0,6
4,Aviron Bayonnais,38,14,9,0,5
5,Stade Rochelais,36,14,8,0,6
6,Montpellier Hérault Rugby,34,14,7,0,7
7,Castres Olympique,31,14,7,0,7
8,Racing 92,26,14,5,1,8
9,LOU Rugby,26,14,5,2,7


In [264]:
# regex_nb_matchs = re.search(r'[0-9][0-9]', str(nb_matchs[4:8][0]))
# regex_nb_matchs[0]

In [265]:
# for team in range(0, 2):    
#     for matchs in nb_matchs[(4*(team+1)):(4*(team+1)+4)]:
#         # print(matchs)
#         matchs = str(matchs)
#         # print(matchs[0])
#         regex_matchs = re.search(r'[0-9]+', matchs)
#         print(regex_matchs[0])

In [266]:
# list_nb_matchs = []
# for ele in range(0,4):
#     for matchs in nb_matchs[4:8][ele]:
#         matchs = str(matchs)
#         # print(matchs)
#         regex_nb_matchs = re.search(r'[0-9][0-9]', str(nb_matchs[4:8]))
#         print(regex_nb_matchs[0])
        

- affichage du df final

In [267]:
# df_classement_general = pd.DataFrame({
#                                         "Equipes": list_team,
#                                         "Points": list_points_team
# })

In [268]:
# df_classement_general

In [269]:
# url = "https://top14.lnr.fr/classement/domicile"

# def classement_general(url):
    
#     page = requests.get(url)
#     # return(page.text)
    
#     soup = BeautifulSoup(page.text, 'html.parser')
#     # return(soup)
    
#     list_nom_equipes = soup.find_all('a', {'class': 'base-link base-link--black'})
#     # return(list_nom_equipes)
    
#     list_team = []
#     for ele in list_nom_equipes:
#         ele = str(ele)
#         # Appliquer la regex à chaque élément `ele`
#         match = re.search(r'>([^<]+)<', ele)
#         if match:  # Vérifier si la regex a trouvé un résultat
#             list_team.append(match.group(1).strip())

#     # Afficher le résultat final
#     # return(list_team)
    
#     list_points_team = []
#     for points in points_equipes:
#         # print(points)
#         points = str(points)
#         regex_points = re.search(r'[0-9][0-9]', points)
#         # print(regex_points)
#         regex_points[0]
#         # print(regex_points[0])
#         list_points_team.append(regex_points[0])

#     # return(list_points_team)
    
#     return(pd.DataFrame({
        
#                 "Equipes":list_team,
#                 "Pts": list_points_team
#     }))

In [270]:
# classement_general(url)

In [271]:
# urls = ["https://top14.lnr.fr/classement", "https://top14.lnr.fr/classement/domicile", "https://top14.lnr.fr/classement/exterieur"]

In [272]:
# for url in urls:
#     display(classement_general(url))
#     print('---')